### MNIST

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-06-19 10:30:46.689016: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-19 10:30:46.689116: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label


In [4]:
scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

In [5]:
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

In [6]:
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

In [7]:
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [8]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)

In [9]:
validation_data = validation_data.batch(num_validation_samples)

In [10]:
test_data = scaled_test_data.batch(num_test_samples)

In [11]:
validation_inputs, validation_targets = next(iter(validation_data))

2023-06-19 10:30:46.879375: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2023-06-19 10:30:47.227363: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


#### Model

Outline the model

In [30]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

Choose the optimizer and loss function

In [31]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Training

In [32]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5


2023-06-19 10:46:47.766698: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-19 10:46:51.644221: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


540/540 - 5s - loss: 0.2654 - accuracy: 0.9208 - val_loss: 0.1203 - val_accuracy: 0.9653 - 5s/epoch - 9ms/step
Epoch 2/5
540/540 - 4s - loss: 0.1003 - accuracy: 0.9691 - val_loss: 0.0825 - val_accuracy: 0.9742 - 4s/epoch - 8ms/step
Epoch 3/5
540/540 - 4s - loss: 0.0667 - accuracy: 0.9786 - val_loss: 0.0707 - val_accuracy: 0.9753 - 4s/epoch - 8ms/step
Epoch 4/5
540/540 - 4s - loss: 0.0516 - accuracy: 0.9836 - val_loss: 0.0439 - val_accuracy: 0.9872 - 4s/epoch - 8ms/step
Epoch 5/5
540/540 - 4s - loss: 0.0403 - accuracy: 0.9871 - val_loss: 0.0423 - val_accuracy: 0.9870 - 4s/epoch - 8ms/step


We get an accuracy of 98.7% which is very good!